This notebook contains the source code for generating the rectangular mask for notehead based 
on its vertical position. A notehead whose vertical position is on the bottom-most line of the staff 
is quantized to 0th bin. A notehead whose vertical position is above the bottom-most line will have 
negative quantized bin, and, similarly, a notehead whose vertical position is below the bottom-most line
will have positive quantized bin.

In producing the masks, any sheet music that contains a staff with only one line is discarded.

In [1]:
from sheet_id.utils.base_utils import loadSettings
from glob import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
import xml.etree.ElementTree
import tqdm

from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

In [2]:
settings = loadSettings()

In [3]:
data = {}
base_path = settings['DEEPSCORES_PATH']
for path in glob(os.path.join(base_path, 'images_png', '*.png')):
    filename = os.path.splitext(os.path.split(path)[1])[0]
    data[filename] = {
        'png': path,
        'annotation': path.replace('/images_png/', '/pix_annotations_png/'),
        'xml': path.replace('/images_png/', '/xml_annotations/').replace('.png', '.xml'),
    }

In [4]:
def calcCenter(obj):
    return ((obj[0]+obj[2])/2, (obj[1]+obj[3])/2)

def calcCoord(obj, width, height):
    bbox = obj.find('bndbox')
    x1 = (float(bbox.find('xmin').text)*width)
    y1 = (float(bbox.find('ymin').text)*height)
    x2 = (float(bbox.find('xmax').text)*width)
    y2 = (float(bbox.find('ymax').text)*height)
    return (x1,y1,x2,y2)

def calcYdist(coord1, coord2):
    return abs(coord2[1] - coord1[1])

def calcDim(obj):
    return (obj[2] - obj[0], obj[3] - obj[1])

In [5]:
def plot(img, boxes, figsize=(12,12), show=True):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.imshow(img, cmap='gray')

    patches = []
    for (box,name) in boxes:
        patches.append(Rectangle((box[0],box[1]), box[2]-box[0], box[3]-box[1], label=name))
        ax.text(box[0], box[1], name)
    pc = PatchCollection(patches, facecolor='None', edgecolor='r')
    ax.add_collection(pc)
    if show: plt.show()

In [6]:
conversion = {
    ('gClef', 'emmentaler'): 47.802,
    ('fClef', 'emmentaler'): 57.062,
    ('cClefAlto', 'emmentaler'): 46.416,
    ('cClefTenor', 'emmentaler'): 69.052,
    ('unpitchedPercussionClef1', 'emmentaler'): 46.269,
    ('gClef', 'lilyjazz'): 43.046,
    ('fClef', 'lilyjazz'): 44.507,
    ('cClefAlto', 'lilyjazz'): 48.920,
    ('cClefTenor', 'lilyjazz'): 67.891,
    ('unpitchedPercussionClef1', 'lilyjazz'): 45.638,
    ('gClef', 'gonville'): 44.429,
    ('fClef', 'gonville'): 53.245,
    ('cClefAlto', 'gonville'): 45.383,
    ('cClefTenor', 'gonville'): 66.697,
    ('unpitchedPercussionClef1', 'gonville'): 46.435,
    ('gClef', 'gutenberg1939'): 43.765,
    ('fClef', 'gutenberg1939'): 57.461,
    ('cClefAlto', 'gutenberg1939'): 48.548,
    ('cClefTenor', 'gutenberg1939'): 67.082,
    ('unpitchedPercussionClef1', 'gutenberg1939'): 49.750,
    ('gClef', 'beethoven'): 52.048,
    ('fClef', 'beethoven'): 54.837,
    ('cClefAlto', 'beethoven'): 46.518,
    ('cClefTenor', 'beethoven'): 71.787,
    ('unpitchedPercussionClef1', 'beethoven'): 46.691,
}
noteheadSize = {
    'emmentaler': 23,
    'lilyjazz': 23,
    'gonville': 23,
    'gutenberg1939': 23,
    'beethoven': 23,
}

def discretizeNotehead(coordNotehead, yCoordClef, clef, font):
    noteYCenter = calcCenter(coordNotehead)[1]
    yCoordClef += conversion[(clef, font)]
    distBetweenBins = noteheadSize[font] / 2
    quantized = (round((noteYCenter - yCoordClef) / (distBetweenBins / 2)) + 1) // 2
    if quantized < -15 or quantized > 10:
        raise ValueError('quantized bin is too far from the staff (expeceted <= 10 and >= -15, actual {:})'.format(quantized))
    return quantized

In [7]:
noteheadClass = ['noteheadBlack', 'noteheadHalf', 'noteheadWhole']
clefClass = ['gClef', 'fClef', 'cClefAlto', 'cClefTenor', 'unpitchedPercussionClef1']

In [8]:
unprocessed_keys = []
keys = sorted(list(data.keys()))
for key in tqdm.tqdm_notebook(keys):
    error = False

    font = key.split('-')[3]
    tree = xml.etree.ElementTree.parse(data[key]['xml'])
    objs = tree.findall('object')

    # Get the size of the entire sheet
    size = tree.find('size')
    height, width = float(size.find('height').text), float(size.find('width').text)

    clefPosition = []
    lines = []

    # Find all clefs
    for ix, obj in enumerate(objs):
        name = obj.find('name').text
        if name in ['gClefChange', 'cClefAltoChange', 'fClefChange', 'cClefTenorChange']: continue
        if 'Clef' in name: # name in clefClass:
            coord = calcCoord(obj, width, height)
            clefPosition.append((calcCenter(coord), name))
            lines.append([])
    clefPosition = sorted(clefPosition, key=lambda x: x[0][1]) # sort by vertical position
    
    # Group notehead based on clefs    
    for ix, obj in enumerate(objs):
        name = obj.find('name').text
        if name in noteheadClass:
            coord = calcCoord(obj, width, height)
            mindist = np.inf
            bestAnswer = -1
            for i in range(len(clefPosition)):
                dist = calcYdist(coord, clefPosition[i][0])
                if dist < mindist:
                    mindist = dist
                    bestAnswer = i
            try:
                quantized_note = discretizeNotehead(coord, clefPosition[bestAnswer][0][1], clefPosition[bestAnswer][1], font)
                obj.find('name').text = name + '_' + str(quantized_note) 
            except (IndexError, ValueError) as e:
                unprocessed_keys.append(key)
                print(key)
                error = True
                break
            
    if not error:
        newpath = data[key]['xml'].replace('/xml_annotations/', '/xml_annotations_new/')
        tree.write(newpath)

unprocessed_keys = set(unprocessed_keys)

HBox(children=(IntProgress(value=0, max=210000), HTML(value='')))

lg-100052827-aug-beethoven--page-1
lg-100052827-aug-beethoven--page-2
lg-100052827-aug-beethoven--page-3
lg-100052827-aug-beethoven--page-4
lg-100052827-aug-emmentaler--page-1
lg-100052827-aug-emmentaler--page-2
lg-100052827-aug-emmentaler--page-3
lg-100052827-aug-emmentaler--page-4
lg-100052827-aug-gonville--page-1
lg-100052827-aug-gonville--page-2
lg-100052827-aug-gonville--page-3
lg-100052827-aug-gonville--page-4
lg-100052827-aug-gonville--page-5
lg-100052827-aug-gutenberg1939--page-1
lg-100052827-aug-gutenberg1939--page-2
lg-100052827-aug-gutenberg1939--page-3
lg-100052827-aug-gutenberg1939--page-4
lg-100052827-aug-gutenberg1939--page-5
lg-100052827-aug-lilyjazz--page-1
lg-100052827-aug-lilyjazz--page-3
lg-100080340-aug-beethoven--page-1
lg-100080340-aug-beethoven--page-10
lg-100080340-aug-beethoven--page-2
lg-100080340-aug-beethoven--page-5
lg-100080340-aug-beethoven--page-6
lg-100080340-aug-beethoven--page-7
lg-100080340-aug-beethoven--page-8
lg-100080340-aug-beethoven--page-9
lg

lg-102056465-aug-gonville--page-10
lg-102056465-aug-gonville--page-12
lg-102056465-aug-gonville--page-14
lg-102056465-aug-gonville--page-20
lg-102056465-aug-gonville--page-21
lg-102056465-aug-gonville--page-26
lg-102056465-aug-gonville--page-27
lg-102056465-aug-gonville--page-29
lg-102056465-aug-gonville--page-30
lg-102056465-aug-gonville--page-31
lg-102056465-aug-gonville--page-36
lg-102056465-aug-gonville--page-5
lg-102056465-aug-gonville--page-6
lg-102056465-aug-gonville--page-7
lg-102056465-aug-gutenberg1939--page-12
lg-102056465-aug-gutenberg1939--page-14
lg-102056465-aug-gutenberg1939--page-15
lg-102056465-aug-gutenberg1939--page-16
lg-102056465-aug-gutenberg1939--page-19
lg-102056465-aug-gutenberg1939--page-20
lg-102056465-aug-gutenberg1939--page-26
lg-102056465-aug-gutenberg1939--page-3
lg-102056465-aug-gutenberg1939--page-30
lg-102056465-aug-gutenberg1939--page-33
lg-102056465-aug-gutenberg1939--page-5
lg-102056465-aug-gutenberg1939--page-6
lg-102056465-aug-gutenberg1939--page

lg-103475154-aug-beethoven--page-2
lg-103475154-aug-emmentaler--page-2
lg-103475154-aug-emmentaler--page-3
lg-103475154-aug-gonville--page-3
lg-103475154-aug-gutenberg1939--page-2
lg-103475154-aug-gutenberg1939--page-3
lg-103608116-aug-beethoven--page-1
lg-103608116-aug-beethoven--page-2
lg-103608116-aug-emmentaler--page-1
lg-103608116-aug-gonville--page-1
lg-103608116-aug-gonville--page-2
lg-103608116-aug-gutenberg1939--page-1
lg-103608116-aug-gutenberg1939--page-2
lg-103608116-aug-lilyjazz--page-1
lg-103634947-aug-emmentaler--page-33
lg-103634947-aug-lilyjazz--page-33
lg-103934383-aug-beethoven--page-1
lg-103934383-aug-beethoven--page-11
lg-103934383-aug-beethoven--page-21
lg-103934383-aug-beethoven--page-3
lg-103934383-aug-beethoven--page-32
lg-103934383-aug-beethoven--page-36
lg-103934383-aug-beethoven--page-5
lg-103934383-aug-beethoven--page-8
lg-103934383-aug-emmentaler--page-1
lg-103934383-aug-emmentaler--page-11
lg-103934383-aug-emmentaler--page-12
lg-103934383-aug-emmentaler--

lg-105016670422551128-aug-lilyjazz--page-7
lg-105016670422551128-aug-lilyjazz--page-9
lg-10511152560816632-aug-beethoven-
lg-10511152560816632-aug-gutenberg1939-
lg-10511152560816632-aug-lilyjazz-
lg-105118553243166875-aug-beethoven--page-1
lg-105118553243166875-aug-beethoven--page-10
lg-105118553243166875-aug-beethoven--page-12
lg-105118553243166875-aug-beethoven--page-14
lg-105118553243166875-aug-beethoven--page-16
lg-105118553243166875-aug-beethoven--page-17
lg-105118553243166875-aug-beethoven--page-18
lg-105118553243166875-aug-beethoven--page-2
lg-105118553243166875-aug-beethoven--page-20
lg-105118553243166875-aug-beethoven--page-22
lg-105118553243166875-aug-beethoven--page-23
lg-105118553243166875-aug-beethoven--page-24
lg-105118553243166875-aug-beethoven--page-4
lg-105118553243166875-aug-beethoven--page-5
lg-105118553243166875-aug-beethoven--page-7
lg-105118553243166875-aug-beethoven--page-8
lg-105118553243166875-aug-beethoven--page-9
lg-105118553243166875-aug-emmentaler--page-1


...

lg-91688491-aug-beethoven--page-24
lg-91688491-aug-beethoven--page-36
lg-91688491-aug-gonville--page-22
lg-91688491-aug-gonville--page-23
lg-91688491-aug-gonville--page-35
lg-91688491-aug-gutenberg1939--page-14
lg-91688491-aug-gutenberg1939--page-22
lg-91688491-aug-gutenberg1939--page-23
lg-91688491-aug-gutenberg1939--page-35
lg-91688491-aug-lilyjazz--page-26
lg-91688491-aug-lilyjazz--page-38
lg-916979230585598927-aug-beethoven--page-10
lg-916979230585598927-aug-beethoven--page-11
lg-916979230585598927-aug-beethoven--page-12
lg-916979230585598927-aug-beethoven--page-13
lg-916979230585598927-aug-beethoven--page-3
lg-916979230585598927-aug-beethoven--page-4
lg-916979230585598927-aug-beethoven--page-6
lg-916979230585598927-aug-beethoven--page-7
lg-916979230585598927-aug-beethoven--page-8
lg-916979230585598927-aug-emmentaler--page-1
lg-916979230585598927-aug-emmentaler--page-10
lg-916979230585598927-aug-emmentaler--page-11
lg-916979230585598927-aug-emmentaler--page-12
lg-916979230585598927

lg-93432157-aug-beethoven--page-10
lg-93432157-aug-beethoven--page-12
lg-93432157-aug-beethoven--page-16
lg-93432157-aug-beethoven--page-18
lg-93432157-aug-beethoven--page-19
lg-93432157-aug-beethoven--page-2
lg-93432157-aug-beethoven--page-9
lg-93432157-aug-emmentaler--page-12
lg-93432157-aug-emmentaler--page-13
lg-93432157-aug-emmentaler--page-16
lg-93432157-aug-emmentaler--page-19
lg-93432157-aug-emmentaler--page-2
lg-93432157-aug-emmentaler--page-20
lg-93432157-aug-emmentaler--page-21
lg-93432157-aug-emmentaler--page-3
lg-93432157-aug-emmentaler--page-4
lg-93432157-aug-emmentaler--page-5
lg-93432157-aug-emmentaler--page-7
lg-93432157-aug-gonville--page-10
lg-93432157-aug-gonville--page-12
lg-93432157-aug-gonville--page-13
lg-93432157-aug-gonville--page-14
lg-93432157-aug-gonville--page-15
lg-93432157-aug-gonville--page-19
lg-93432157-aug-gonville--page-2
lg-93432157-aug-gonville--page-21
lg-93432157-aug-gonville--page-3
lg-93432157-aug-gonville--page-4
lg-93432157-aug-gonville--pag

lg-95023075-aug-emmentaler--page-3
lg-95023075-aug-gonville--page-2
lg-95023075-aug-gonville--page-3
lg-95023075-aug-gutenberg1939--page-2
lg-95023075-aug-lilyjazz--page-1
lg-95023075-aug-lilyjazz--page-2
lg-95023075-aug-lilyjazz--page-3
lg-95127144-aug-beethoven--page-1
lg-95127144-aug-beethoven--page-10
lg-95127144-aug-beethoven--page-11
lg-95127144-aug-beethoven--page-12
lg-95127144-aug-beethoven--page-14
lg-95127144-aug-beethoven--page-15
lg-95127144-aug-beethoven--page-16
lg-95127144-aug-beethoven--page-2
lg-95127144-aug-beethoven--page-22
lg-95127144-aug-beethoven--page-23
lg-95127144-aug-beethoven--page-25
lg-95127144-aug-beethoven--page-26
lg-95127144-aug-beethoven--page-3
lg-95127144-aug-beethoven--page-34
lg-95127144-aug-beethoven--page-35
lg-95127144-aug-beethoven--page-37
lg-95127144-aug-beethoven--page-4
lg-95127144-aug-beethoven--page-5
lg-95127144-aug-beethoven--page-6
lg-95127144-aug-beethoven--page-8
lg-95127144-aug-emmentaler--page-10
lg-95127144-aug-emmentaler--page-

lg-97319284-aug-beethoven--page-1
lg-97319284-aug-emmentaler--page-1
lg-97319284-aug-emmentaler--page-3
lg-97319284-aug-gonville--page-1
lg-97319284-aug-gonville--page-3
lg-97319284-aug-lilyjazz--page-1
lg-97319284-aug-lilyjazz--page-2
lg-97319284-aug-lilyjazz--page-3
lg-97487894-aug-beethoven--page-10
lg-97487894-aug-beethoven--page-4
lg-97487894-aug-emmentaler--page-10
lg-97487894-aug-emmentaler--page-4
lg-97487894-aug-gonville--page-4
lg-97487894-aug-gutenberg1939--page-10
lg-97487894-aug-lilyjazz--page-10
lg-97652006-aug-beethoven--page-11
lg-97652006-aug-beethoven--page-17
lg-97652006-aug-beethoven--page-19
lg-97652006-aug-beethoven--page-22
lg-97652006-aug-beethoven--page-23
lg-97652006-aug-beethoven--page-3
lg-97652006-aug-beethoven--page-4
lg-97652006-aug-beethoven--page-7
lg-97652006-aug-beethoven--page-8
lg-97652006-aug-beethoven--page-9
lg-97652006-aug-emmentaler--page-13
lg-97652006-aug-emmentaler--page-14
lg-97652006-aug-emmentaler--page-17
lg-97652006-aug-emmentaler--page

lg-98930236-aug-gutenberg1939--page-5
lg-98930236-aug-gutenberg1939--page-6
lg-98930236-aug-lilyjazz--page-3
lg-98930236-aug-lilyjazz--page-5
lg-98930236-aug-lilyjazz--page-6
lg-99299726-aug-beethoven--page-13
lg-99299726-aug-beethoven--page-14
lg-99299726-aug-beethoven--page-5
lg-99299726-aug-beethoven--page-9
lg-99299726-aug-emmentaler--page-13
lg-99299726-aug-emmentaler--page-15
lg-99299726-aug-emmentaler--page-8
lg-99299726-aug-emmentaler--page-9
lg-99299726-aug-gonville--page-13
lg-99299726-aug-gonville--page-14
lg-99299726-aug-gonville--page-5
lg-99299726-aug-gonville--page-8
lg-99299726-aug-gutenberg1939--page-14
lg-99299726-aug-gutenberg1939--page-15
lg-99299726-aug-gutenberg1939--page-9
lg-99299726-aug-lilyjazz--page-14
lg-99299726-aug-lilyjazz--page-15
lg-99299726-aug-lilyjazz--page-5
lg-99299726-aug-lilyjazz--page-9
lg-99331023-aug-beethoven--page-1
lg-99331023-aug-emmentaler--page-1
lg-99606038-aug-beethoven--page-1
lg-99606038-aug-beethoven--page-2
lg-99606038-aug-emmental

In [9]:
len(unprocessed_keys)

36139

In [ ]:
offset = 16
for key in tqdm.tqdm_notebook(keys):
    if key in unprocessed_keys: continue
    
    # Parse XML tree
    tree = xml.etree.ElementTree.parse(data[key]['xml'].replace('/xml_annotations/', '/xml_annotations_new/'))
    
    # Get the size of the entire sheet
    size = tree.find('size')
    height, width = float(size.find('height').text), float(size.find('width').text)
    
    # Create an image placeholder
    img = np.zeros((int(height), int(width)))
    
    # Create a mask for all objects
    objs = tree.findall('object')
    for idx, obj in enumerate(objs):
        name = obj.find('name').text
        if 'notehead' in name and '_' in name:
            quantized_bin = int(name.split('_')[-1])
            coord = calcCoord(obj, width, height)
            img[int(coord[1]):int(coord[3]), int(coord[0]):int(coord[2])] = quantized_bin + offset
    
    # save image
    cv2.imwrite(data[key]['annotation'].replace('/pix_annotations_png/', '/pix_annotations_png_new/'), img)

HBox(children=(IntProgress(value=0, max=210000), HTML(value='')))

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(cv2.imread(data[key]['annotation'].replace('/pix_annotations_png/', '/pix_annotations_png_new/'), 0))
plt.subplot(1,2,2)
plt.imshow(cv2.imread(data[key]['png']))

In [ ]:
key = keys[-20]
img = cv2.imread(data[key]['annotation'].replace('/pix_annotations_png/', '/pix_annotations_png_new/'), 0)
print(img.shape)

In [ ]:
np.unique(img)

In [ ]:
img.shape